In [ ]:
!pip install ragas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.3/284.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.5/449.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.0/352.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: jiter
    Found existing

In [ ]:
# !pip install faiss-cpu

In [ ]:
!pip install pymilvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install milvus-lite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 11.0 MB/s eta 0:00:00


In [ ]:
# Load all required Libraries
import pandas as pd
import transformers, torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

import os
import sys
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from pymilvus import MilvusClient, FieldSchema, CollectionSchema, DataType


# Read Passages from the Datasets and Drop rows if they are NA or empty

In [228]:
passages = pd.read_parquet("hf://datasets/rag-datasets/rag-mini-wikipedia/data/passages.parquet/part.0.parquet")

print(passages.shape)
passages.head()

(3200, 1)


,passage
id,
0,"Uruguay (official full name in ; pron. , Eas..."
1,"It is bordered by Brazil to the north, by Arge..."
2,Montevideo was founded by the Spanish in the e...
3,The economy is largely based in agriculture (m...
4,"According to Transparency International, Urugu..."


# Do EDA on the passage dataset
- You can try to find the maximum and minimum length of the passages before indexing (just a direction)

In [ ]:
# This is in a different notebook

# Tokenize Text and Generate Embeddings using Sentence Transformers

In [229]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

texts = passages["passage"].dropna().astype(str).tolist()

# Encode Text
embeddings = embedding_model.encode(
    texts,
    convert_to_numpy=True,
    show_progress_bar=True
).astype("float32")

print("Embeddings shape:", embeddings.shape)
print("Example vector (first 5 dims):", embeddings[0][:5])

Batches:   0%|          | 0/100 [00:00<?, ?it/s]

Embeddings shape: (3200, 384)
Example vector (first 5 dims): [ 0.00698539 -0.06149816 -0.06683705 -0.00828594  0.04050035]


# Create Milvus Client and Insert your Embeddings to your DB
- Make sure you define a schema for your collection (Points will be deducted if you fail to define a proper schema with ids, passage text, embedding)

In [230]:
# Define every column of your schema

id_ = FieldSchema(
    name="id",
    dtype=DataType.INT64,
    is_primary=True,
    auto_id=False
)
passage_field = FieldSchema(
    name="passage",
    dtype=DataType.VARCHAR,
    max_length=2000
)

embedding_field = FieldSchema(
    name="embedding",
    dtype=DataType.FLOAT_VECTOR,
    dim=embeddings.shape[1]   # 384 for all-MiniLM-L6-v2
)

In [231]:
schema = CollectionSchema(
    fields=[id_, passage_field, embedding_field],
    description="Mini Wikipedia passages with embeddings"
)

In [232]:
client = MilvusClient("rag_wikipedia_mini.db")

# Create the Collection with Collection Name = "rag_mini". Make sure you define the schema variable while creating the collection
try:
    client.drop_collection("rag_mini")
except:
    pass

client.create_collection(
    collection_name="rag_mini",
    schema=schema
)

**Convert your Pandas Dataframe to a list of dictionaries**
- The Dictionary at least have 3 keys [id, passage, embedding]

In [233]:
rag_data = []
for i, (idx, passage_text) in enumerate(passages["passage"].dropna().items()):
    vec = embeddings[i].astype("float32").ravel().tolist()
    vec = [float(x) for x in vec]
    rag_data.append({
        "id": int(idx),
        "passage": passage_text,
        "embedding": vec
    })

In [234]:
# Code to insert the data to your DB
res = client.insert(collection_name="rag_mini", data=rag_data)

print(res)

{'insert_count': 3200, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

- Do a Sanity Check on your database

**Do not delete the below line during your submission**

In [235]:
print("Entity count:", client.get_collection_stats("rag_mini")["row_count"])
print("Collection schema:", client.describe_collection("rag_mini"))

Entity count: 3200
Collection schema: {'collection_name': 'rag_mini', 'auto_id': False, 'num_shards': 0, 'description': 'Mini Wikipedia passages with embeddings', 'fields': [{'field_id': 100, 'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'params': {}, 'is_primary': True}, {'field_id': 101, 'name': 'passage', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'field_id': 102, 'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'functions': [], 'aliases': [], 'collection_id': 0, 'consistency_level': 0, 'properties': {}, 'num_partitions': 0, 'enable_dynamic_field': False}


# Steps to Fetch Results
- Read the Question Dataset
- Clean the Question Dataset if necessary (Drop Questions with NaN etc.)
- Convert Each Query to a Vector Embedding (Use the same embedding model you used to embed your document)
- Try for a Single Question First
- Load Collection into Memory after creating Index for Search on your embedding field (This is an essential step before you can search in your db)
- Search and Fetch Top N Results

In [236]:
import pandas as pd

queries = pd.read_parquet("hf://datasets/rag-datasets/rag-mini-wikipedia/data/test.parquet/part.0.parquet")
queries

,question,answer
id,,
0,Was Abraham Lincoln the sixteenth President of...,yes
2,Did Lincoln sign the National Banking Act of 1...,yes
4,Did his mother die of pneumonia?,no
6,How many long was Lincoln's formal education?,18 months
8,When did Lincoln begin his political career?,1832
...,...,...
1710,Was Wilson president of the American Political...,Yes
1711,Did he not cast his ballot for John M. Palmer ...,Yes
1712,Did Wilson not spend 1914 through the beginnin...,Yes


In [ ]:
query = queries["question"].dropna().iloc[20]                 # Your single query

query_embedding = embedding_model.encode(
    [query],
    convert_to_numpy=True,
    show_progress_bar=False
).astype("float32")

print(query_embedding.shape)

(1, 384)


#### Create Index on the embedding column on your DB

In [237]:
index_params = MilvusClient.prepare_index_params()

# Add an index on the embedding field
index_params.add_index(
    field_name="embedding",
    index_type="IVF_FLAT",
    metric_type="COSINE",
    params={"nlist": 128}
)

# Create the index
try:
    client.create_index(
        collection_name="rag_mini",
        index_params=index_params
    )
    print("Index created successfully!")
except Exception as e:
    print(f"Index creation result: {e}")

# Load collection into memory (required for search)
client.load_collection("rag_mini")

print("Collection loaded into memory")

Index created successfully!
Collection loaded into memory


In [ ]:
# Search the db with your query embedding
qvec = query_embedding.reshape(-1).tolist()
qvec = [float(x) for x in qvec]
assert len(qvec) == 384, f"unexpected query dim: {len(qvec)}"

output_ = client.search(
    collection_name="rag_mini",
    data=[qvec],
    anns_field="embedding",
    search_params={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=3,
    output_fields=["id", "passage"]
)

print(output_)

data: [[{'id': 317, 'distance': 0.690296471118927, 'entity': {'id': 317, 'passage': 'Entering the presidential nomination process as a distinct underdog, Lincoln was eventually chosen as the Republican candidate for the 1860 election for several reasons. His expressed views on slavery were seen as more moderate than those of rivals William H. Seward and Salmon P. Chase. His "Western" origins also appealed to the newer states: other contenders, especially those with more governmental experience, had acquired enemies within the party and were weak in the critical western states, while Lincoln was perceived as a moderate who could win the West. Most Republicans agreed with Lincoln that the North was the aggrieved party as the Slave Power tightened its grasp on the national government. Yet despite his Southern connections (his in-laws owned slaves), Lincoln misunderstood the depth of the revolution underway in the South and the emergence of Southern nationalism. Throughout the 1850s he den

## Now get the Context
- Initially use the first passage ONLY as your context
- In Later Experiments, you must try at least 2 different passage selection strategies (Top 3 / Top 5 / Top 10) and pass to your prompt

In [ ]:
context = output_[0][0]["entity"]["passage"]

**Develop your Prompt**

In [ ]:
system_prompt = f"You are a helpful assistant. Answer the question using ONLY the given context. If the context is insufficient, say 'I don’t know'."

prompt = f"""{system_prompt} \n Context: {context}: \n Question: {query} """
print(prompt)

You are a helpful assistant. Answer the question using ONLY the given context. If the context is insufficient, say 'I don’t know'. 
 Context: Entering the presidential nomination process as a distinct underdog, Lincoln was eventually chosen as the Republican candidate for the 1860 election for several reasons. His expressed views on slavery were seen as more moderate than those of rivals William H. Seward and Salmon P. Chase. His "Western" origins also appealed to the newer states: other contenders, especially those with more governmental experience, had acquired enemies within the party and were weak in the critical western states, while Lincoln was perceived as a moderate who could win the West. Most Republicans agreed with Lincoln that the North was the aggrieved party as the Slave Power tightened its grasp on the national government. Yet despite his Southern connections (his in-laws owned slaves), Lincoln misunderstood the depth of the revolution underway in the South and the emerg

# RAG Response for a Single Query

In [ ]:
# Load the LLM Model you want to use
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

rag_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [ ]:
# Generate answer
result = rag_pipeline(prompt, max_length=200, temperature=0.0)

# Decode and extract answer.
answer = result[0]["generated_text"]

print("RAG Answer:", answer)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


RAG Answer: Lincoln was eventually chosen as the Republican candidate for the 1860 election for several reasons.


# Generate Responses for all the Queries in the Dataset

In [238]:
# Your Code Here
eval_df = queries

STRATEGIES = {
    "instruction": (
        "Answer the following question using ONLY the given context."
        "Do NOT add outside knowledge."
        "Provide the answer in one clear, concise sentence."
        "If the context is insufficient, reply exactly: I don't know."
        "Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    ),
    "cot": (
        "Let's think step by step. First, read the question and context and extract the key entities, numbers, dates, and relations."
        "Then, do a step-by-step reasoning, using ONLY the given context and no outside knowledge. "
        "If the context is insufficient to answer, reply exactly: I don't know."
        "Finally, output the concise answer in one sentence."
        "Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    ),
    "persona": (
       "You are a historian specializing in U.S. political history."
       "Your expertise ensures that answers are accurate, well-grounded, and concise."
       "Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    ),
}

def build_prompt(strategy, question, context):
    return STRATEGIES[strategy].format(question=question, context=context)


In [239]:
if 'id' not in eval_df.columns:
    eval_df = eval_df.reset_index(drop=True)
    eval_df['id'] = eval_df.index.astype(str)
eval_df.head()

,question,answer,id
0,Was Abraham Lincoln the sixteenth President of...,yes,0
1,Did Lincoln sign the National Banking Act of 1...,yes,1
2,Did his mother die of pneumonia?,no,2
3,How many long was Lincoln's formal education?,18 months,3
4,When did Lincoln begin his political career?,1832,4


In [240]:
def retrieve_top1_context(question):
    qvec = embedding_model.encode([question], convert_to_numpy=True, show_progress_bar=False).astype("float32")
    qlist = [float(x) for x in qvec.reshape(-1).tolist()]
    output_ = client.search(
        collection_name="rag_mini",
        data=[qlist],
        anns_field="embedding",
        search_params={"metric_type": "COSINE", "params": {"nprobe": 10}},
        limit=1,
        output_fields=["id", "passage"],
    )
    return output_[0][0]["entity"]["passage"]

In [241]:
from tqdm.auto import tqdm
import time

all_preds = {}

for strategy in STRATEGIES.keys():
    preds = []
    print(f"Running strategy: {strategy}")
    for _, row in tqdm(eval_df.iterrows(), total=len(eval_df), desc=f"{strategy}"):
        q_id = str(row["id"])
        q = str(row["question"])
        context = retrieve_top1_context(q)
        prompt = build_prompt(strategy, q, context)
        out = rag_pipeline(
            prompt,
            max_new_tokens=128,
            do_sample=False,
            num_beams=1
        )[0]["generated_text"].strip()
        preds.append({"id": q_id, "prediction_text": out})
    all_preds[strategy] = preds

references = [
    {"id": str(r["id"]), "answers": {"text": [str(r["answer"])], "answer_start": [0]}}
    for _, r in eval_df.iterrows()
]

print("Done. Strategies:", list(all_preds.keys()), "| #items =", len(references))


Running strategy: instruction


instruction:   0%|          | 0/918 [00:00<?, ?it/s]

Running strategy: cot


cot:   0%|          | 0/918 [00:00<?, ?it/s]

Running strategy: persona


persona:   0%|          | 0/918 [00:00<?, ?it/s]

Done. Strategies: ['instruction', 'cot', 'persona'] | #items = 918


# Finding out the Basic QA Metrics (F1 score, EM score)

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00


In [242]:
def extract_final_answer(s):
    if "Answer:" in s:
        return s.split("Answer:", 1)[1].strip()
    return s.strip()

for strat, preds in all_preds.items():
    for p in preds:
        p["prediction_text"] = extract_final_answer(p["prediction_text"])

In [243]:
# Your code Here
import evaluate, pandas as pd
metric = evaluate.load("squad")

rows = []
for strat, preds in all_preds.items():
    scores = metric.compute(predictions=preds, references=references)
    rows.append({"strategy": strat, "EM": scores["exact_match"], "F1": scores["f1"]})

result_df = pd.DataFrame(rows).sort_values(["F1","EM"], ascending=False).reset_index(drop=True)
display(result_df)

best = result_df.iloc[0]
print(f"Best strategy: {best['strategy']} | EM={best['EM']:.2f} | F1={best['F1']:.2f}")


,strategy,EM,F1
0,instruction,39.760349,46.509372
1,persona,38.888889,45.161512
2,cot,16.122004,24.725195


Best strategy: instruction | EM=39.76 | F1=46.51


In [244]:
def build_records(eval_df, preds_for_strategy, get_contexts_fn):
    id2pred = {str(p["id"]): p["prediction_text"] for p in preds_for_strategy}

    records = []
    for _, r in eval_df.iterrows():
        q_id = str(r["id"])
        q = str(r["question"])
        gt = str(r["answer"])

        ctxs = get_contexts_fn(q)
        if isinstance(ctxs, str):
            ctxs = [ctxs]

        ans = id2pred.get(q_id, "")
        records.append({
            "question": q,
            "answer": ans,         =
            "contexts": [str(c) for c in ctxs],
            "ground_truth": gt
        })
    return records

In [245]:
# naive：top-1
naive_records = build_records(
    eval_df=eval_df,
    preds_for_strategy=all_preds[BEST_STRATEGY],
    get_contexts_fn=lambda q: [retrieve_top1_context(q)]
)

print(len(naive_records))

918


# Experimentation

In [ ]:
passages = pd.read_parquet("hf://datasets/rag-datasets/rag-mini-wikipedia/data/passages.parquet/part.0.parquet")

print(passages.shape)
passages.head()

(3200, 1)


,passage
id,
0,"Uruguay (official full name in ; pron. , Eas..."
1,"It is bordered by Brazil to the north, by Arge..."
2,Montevideo was founded by the Spanish in the e...
3,The economy is largely based in agriculture (m...
4,"According to Transparency International, Urugu..."


In [ ]:
# Embbeding sizes changed to 384 or 512
model_names = {
    "384d": "sentence-transformers/all-MiniLM-L6-v2",   # 384
    "512d": "sentence-transformers/distiluse-base-multilingual-cased-v1",  # 512
}

texts = passages["passage"].dropna().astype(str).tolist()
embeddings_by_model = {}

for tag, model_name in model_names.items():
    print(f"\n=== Building embeddings with {model_name} ({tag}) ===")
    embedding_model = SentenceTransformer(model_name)

    out_dim = embedding_model.get_sentence_embedding_dimension()
    print(f"Model output dim reported by ST: {out_dim}")

    embeddings = embedding_model.encode(
        texts,
        convert_to_numpy=True,
        show_progress_bar=True,
    ).astype("float32")

    print("Embeddings shape:", embeddings.shape)
    print("Example vector (first 5 dims):", embeddings[0][:5])

    embeddings_by_model[tag] = embeddings

    np.save(f"embeddings_{out_dim}d.npy", embeddings)

dims_info = {tag: arr.shape[1] for tag, arr in embeddings_by_model.items()}
print("\nDims summary:", dims_info)



=== Building embeddings with sentence-transformers/all-MiniLM-L6-v2 (384d) ===
Model output dim reported by ST: 384


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

Embeddings shape: (3200, 384)
Example vector (first 5 dims): [ 0.00698539 -0.06149816 -0.06683705 -0.00828594  0.04050035]

=== Building embeddings with sentence-transformers/distiluse-base-multilingual-cased-v1 (512d) ===
Model output dim reported by ST: 512


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

Embeddings shape: (3200, 512)
Example vector (first 5 dims): [ 0.11039015 -0.02015983 -0.08254571 -0.00307726 -0.03157021]

Dims summary: {'384d': 384, '512d': 512}


In [ ]:
# Define every column of your schema

client = MilvusClient("rag_wikipedia_mini.db")

for tag, emb in embeddings_by_model.items():
    collection_name = f"rag_mini_{tag}"
    dim = int(emb.shape[1])

    id_ = FieldSchema(
        name="id",
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=False
    )

    passage_field = FieldSchema(
        name="passage",
        dtype=DataType.VARCHAR,
        max_length=2000
    )

    embedding_field = FieldSchema(
        name="embedding",
        dtype=DataType.FLOAT_VECTOR,
        dim=dim
    )

    schema = CollectionSchema(
        fields=[id_, passage_field, embedding_field],
        description=f"Mini Wikipedia with {tag} embeddings"
    )

    try:
        client.drop_collection(collection_name)
    except:
        pass

    client.create_collection(
        collection_name=collection_name,
        schema=schema
    )
    print(f"✅ Created collection {collection_name} with dim={emb.shape[1]}")


✅ Created collection rag_mini_384d with dim=384
✅ Created collection rag_mini_512d with dim=512


In [ ]:
# Convert your Pandas Dataframe to a list of dictionaries
for tag, emb in embeddings_by_model.items():
    collection_name = f"rag_mini_{tag}"
    print(f"\n=== Inserting {tag} embeddings into {collection_name} ===")

    rag_data = []
    for i, (idx, passage_text) in enumerate(passages["passage"].dropna().items()):
        vec = emb[i].astype("float32").ravel().tolist()
        vec = [float(x) for x in vec]
        rag_data.append({
            "id": int(idx),
            "passage": passage_text,
            "embedding": vec
        })

    res = client.insert(collection_name=collection_name, data=rag_data)
    print(res)

    index_params = client.prepare_index_params()
    index_params.add_index(
        field_name="embedding",
        index_type="AUTOINDEX",
        metric_type="COSINE"
    )

    client.create_index(collection_name=collection_name, index_params=index_params)

    client.load_collection(collection_name)


    print("Entity count:", client.get_collection_stats(collection_name)["row_count"])
    print("Collection schema:", client.describe_collection(collection_name))



=== Inserting 384d embeddings into rag_mini_384d ===
{'insert_count': 3200, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 

In [ ]:
import pandas as pd

queries = pd.read_parquet("hf://datasets/rag-datasets/rag-mini-wikipedia/data/test.parquet/part.0.parquet")
queries

,question,answer
id,,
0,Was Abraham Lincoln the sixteenth President of...,yes
2,Did Lincoln sign the National Banking Act of 1...,yes
4,Did his mother die of pneumonia?,no
6,How many long was Lincoln's formal education?,18 months
8,When did Lincoln begin his political career?,1832
...,...,...
1710,Was Wilson president of the American Political...,Yes
1711,Did he not cast his ballot for John M. Palmer ...,Yes
1712,Did Wilson not spend 1914 through the beginnin...,Yes


In [ ]:
if 'id' not in queries.columns:
    queries = queries.reset_index(drop=True)
    queries['id'] = queries.index.astype(int)

queries.head()

,question,answer,id
0,Was Abraham Lincoln the sixteenth President of...,yes,0
1,Did Lincoln sign the National Banking Act of 1...,yes,1
2,Did his mother die of pneumonia?,no,2
3,How many long was Lincoln's formal education?,18 months,3
4,When did Lincoln begin his political career?,1832,4


In [ ]:
# Get 102 queries of different complexity levels

qdf = queries[['id','question','answer']].dropna().copy()
qdf['qlen'] = qdf['question'].astype(str).str.len()

q1, q2 = qdf['qlen'].quantile([1/3, 2/3])
def bucket(x):
    if x <= q1:   return 'short'
    if x <= q2:   return 'medium'
    return 'long'
qdf['bucket'] = qdf['qlen'].apply(bucket)

target = {'short':34, 'medium':34, 'long':34}
picked = []
for b, need in target.items():
    g = qdf[qdf['bucket']==b]
    picked.append(g.sample(n=min(need, len(g)), random_state=42))

eval_df = pd.concat(picked)
eval_df = eval_df.sample(frac=1, random_state=42).reset_index(drop=True)

print('Eval size:', len(eval_df))
print(eval_df['bucket'].value_counts())
eval_df.head()


Eval size: 102
bucket
short     34
medium    34
long      34
Name: count, dtype: int64


,id,question,answer,qlen,bucket
0,302,Are wolves built for stamina?,Yes,29,short
1,595,What drives sea otter almost to extinction?,Humans hunted them almost to extinction.,43,medium
2,548,"Did Webster write, ""I can now sleep of nights""?",Yes.,47,medium
3,224,What body of water lies to the south of Finland?,the Gulf of Finland,48,medium
4,661,Is a polar bear at high risk of extinction?,yes,43,medium


In [ ]:
# Your Code Here

STRATEGIES = {
    "instruction": (
        "Answer the following question using ONLY the given context."
        "Do NOT add outside knowledge."
        "Provide the answer in one clear, concise sentence."
        "If the context is insufficient, reply exactly: I don't know."
        "Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    ),
    "cot": (
        "Let's think step by step. First, read the question and context and extract the key entities, numbers, dates, and relations."
        "Then, do a step-by-step reasoning, using ONLY the given context and no outside knowledge. "
        "If the context is insufficient to answer, reply exactly: I don't know."
        "Finally, output the concise answer in one sentence."
        "Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    ),
    "persona": (
       "You are a historian specializing in U.S. political history."
       "Your expertise ensures that answers are accurate, well-grounded, and concise."
       "Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    ),
}

def build_prompt(strategy, question, context):
    return STRATEGIES[strategy].format(question=question, context=context)

In [ ]:
from sentence_transformers import SentenceTransformer

COLLECTIONS = {"384d": "rag_mini_384d", "512d": "rag_mini_512d"}
K_LIST = [1, 3, 5]

QUERY_ENCODERS = {
    "384d": SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2"),  # 384
    "512d": SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v1"),  # 512
}

def retrieve_topk_context(question, k, collection_name, encoder):
    qvec = encoder.encode([question], convert_to_numpy=True, show_progress_bar=False, normalize_embeddings=True).astype("float32")
    qlist = [float(x) for x in qvec.reshape(-1).tolist()]
    output_ = client.search(
        collection_name=collection_name,
        data=[qlist],
        anns_field="embedding",
        search_params={"metric_type": "COSINE", "params": {"nprobe": 10}},
        limit=int(k),
        output_fields=["id", "passage"],
    )
    passages = [h["entity"]["passage"] for h in output_[0]]
    return "\n\n---\n\n".join(passages)


In [ ]:
from tqdm.auto import tqdm
import time

all_preds = {}

for strategy in STRATEGIES.keys():
    for dim_tag, coll in COLLECTIONS.items():
        encoder = QUERY_ENCODERS[dim_tag]
        for k in K_LIST:
            key = f"{strategy}|{dim_tag}|k={k}"
            preds = []
            print(f"\nRunning: {key}")
            for _, row in tqdm(eval_df.iterrows(), total=len(eval_df), desc=key):
                q_id = str(row["id"])
                q = str(row["question"])
                context = retrieve_topk_context(q, k=k, collection_name=coll, encoder=encoder)
                prompt = build_prompt(strategy, q, context)
                out = rag_pipeline(
                    prompt,
                    max_new_tokens=128,
                    do_sample=False,
                    num_beams=1
                )[0]["generated_text"].strip()
                preds.append({"id": q_id, "prediction_text": out})
            all_preds[key] = preds

references = [
    {"id": str(r["id"]), "answers": {"text": [str(r["answer"])], "answer_start": [0]}}
    for _, r in eval_df.iterrows()
]

print("\nDone. Keys:", list(all_preds.keys()))
print("#items per setting should be", len(references))



Running: instruction|384d|k=1


instruction|384d|k=1:   0%|          | 0/102 [00:00<?, ?it/s]


Running: instruction|384d|k=3


instruction|384d|k=3:   0%|          | 0/102 [00:00<?, ?it/s]


Running: instruction|384d|k=5


...


Running: persona|512d|k=3


persona|512d|k=3:   0%|          | 0/102 [00:00<?, ?it/s]


Running: persona|512d|k=5


persona|512d|k=5:   0%|          | 0/102 [00:00<?, ?it/s]


Done. Keys: ['instruction|384d|k=1', 'instruction|384d|k=3', 'instruction|384d|k=5', 'instruction|512d|k=1', 'instruction|512d|k=3', 'instruction|512d|k=5', 'cot|384d|k=1', 'cot|384d|k=3', 'cot|384d|k=5', 'cot|512d|k=1', 'cot|512d|k=3', 'cot|512d|k=5', 'persona|384d|k=1', 'persona|384d|k=3', 'persona|384d|k=5', 'persona|512d|k=1', 'persona|512d|k=3', 'persona|512d|k=5']
#items per setting should be 102


In [ ]:
for strat_key, preds in all_preds.items():
    for p in preds:
        p["prediction_text"] = extract_final_answer(p["prediction_text"])

In [ ]:
import evaluate, pandas as pd
metric = evaluate.load("squad")

rows = []
for strat_key, preds in all_preds.items():
    scores = metric.compute(predictions=preds, references=references)
    parts = strat_key.split("|")
    strategy = parts[0]
    dim_tag  = parts[1] if len(parts) > 1 else "384d"
    k_val    = int(parts[2].split("=")[1]) if len(parts) > 2 else 1

    rows.append({
        "strategy": strategy,
        "dim": dim_tag,
        "topk": k_val,
        "EM": scores["exact_match"],
        "F1": scores["f1"],
    })

result_df = pd.DataFrame(rows).sort_values(["F1","EM"], ascending=False).reset_index(drop=True)
display(result_df)

best = result_df.iloc[0]
print(f"Best setting -> strategy={best['strategy']} | dim={best['dim']} | topk={best['topk']} | EM={best['EM']:.2f} | F1={best['F1']:.2f}")

summary = (result_df
           .pivot_table(index=["dim","topk"], columns="strategy", values="F1", aggfunc="max")
           .sort_index())
display(summary)

,strategy,dim,topk,EM,F1
0,instruction,512d,3,46.078431,52.748977
1,instruction,512d,5,45.098039,52.467216
2,instruction,384d,5,45.098039,51.687815
3,instruction,512d,1,42.156863,51.534651
4,instruction,384d,3,41.176471,49.294793
5,instruction,384d,1,42.156863,49.134725
6,persona,512d,1,41.176471,48.729627
7,persona,384d,3,40.196078,48.127911
8,persona,384d,5,39.215686,47.108558
9,persona,512d,3,38.235294,46.369622


Best setting -> strategy=instruction | dim=512d | topk=3 | EM=46.08 | F1=52.75


strategy         cot  instruction    persona
dim  topk                                   
384d 1     27.309046    49.134725  46.100771
     3     25.911011    49.294793  48.127911
     5     29.785784    51.687815  47.108558
512d 1     26.675944    51.534651  48.729627
     3     27.130134    52.748977  46.369622
     5     27.193741    52.467216  45.533907

# Advanced RAG

In [ ]:
passages = pd.read_parquet("hf://datasets/rag-datasets/rag-mini-wikipedia/data/passages.parquet/part.0.parquet")

print(passages.shape)
passages.head()

(3200, 1)


,passage
id,
0,"Uruguay (official full name in ; pron. , Eas..."
1,"It is bordered by Brazil to the north, by Arge..."
2,Montevideo was founded by the Spanish in the e...
3,The economy is largely based in agriculture (m...
4,"According to Transparency International, Urugu..."


In [ ]:
# ---- A) Sentence Splitting + Sliding Window ----
# Sentence-first chunking: split text into sentences, then slide long sentences into overlapping windows by token count
import re
import tiktoken

ENC = tiktoken.get_encoding("cl100k_base")

def tok_count(s: str) -> int:
    return len(ENC.encode(s))

def split_sentences(text: str):
    text = re.sub(r'([。！？])', r'\1 ', text)
    parts = re.split(r'(?<=[\.\?\!。！？])\s+', text.strip())
    return [p.strip() for p in parts if p.strip()]

def slide_long_sentence(sent: str, win_tokens=180, stride_tokens=140):
    n = tok_count(sent)
    if n <= win_tokens:
        return [sent.strip()]
    toks = ENC.encode(sent)
    out, start = [], 0
    while start < n:
        end = min(start + win_tokens, n)
        out.append(ENC.decode(toks[start:end]).strip())
        if end == n:
            break
        start += stride_tokens
    return out

def chunkify_sentence_first(text: str, max_sent_tokens=220, win_tokens=180, stride_tokens=140):
  # Keep short sentences; break only those exceeding max_sent_tokens using sliding windows
    chunks = []
    for sent in split_sentences(text):
        if tok_count(sent) <= max_sent_tokens:
            chunks.append(sent)
        else:
            chunks.extend(slide_long_sentence(sent, win_tokens, stride_tokens))
    return chunks

all_chunks = []
for raw in passages["passage"].dropna().astype(str).tolist():
    all_chunks.extend(chunkify_sentence_first(raw))

texts = all_chunks
print("Total chunks:", len(texts))


Total chunks: 11255


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# texts = passages["passage"].dropna().astype(str).tolist()

# Encode Text
embeddings = embedding_model.encode(
    texts,
    convert_to_numpy=True,
    show_progress_bar=True
).astype("float32")

print("Embeddings shape:", embeddings.shape)
print("Example vector (first 5 dims):", embeddings[0][:5])

Batches:   0%|          | 0/352 [00:00<?, ?it/s]

Embeddings shape: (11255, 384)
Example vector (first 5 dims): [-0.0646422   0.01812241 -0.08958505 -0.01652388  0.01688569]


## Create Milvus Client and Insert your Embeddings to your DB
- Make sure you define a schema for your collection (Points will be deducted if you fail to define a proper schema with ids, passage text, embedding)

In [ ]:
# Define every column of your schema

id_ = FieldSchema(
    name="id",
    dtype=DataType.INT64,
    is_primary=True,
    auto_id=False
)
passage_field = FieldSchema(
    name="passage",
    dtype=DataType.VARCHAR,
    max_length=2000
)

embedding_field = FieldSchema(
    name="embedding",
    dtype=DataType.FLOAT_VECTOR,
    dim=embeddings.shape[1]   # 384 for all-MiniLM-L6-v2
)

In [ ]:
schema = CollectionSchema(
    fields=[id_, passage_field, embedding_field],
    description="Mini Wikipedia passages with embeddings"
)

In [ ]:
client = MilvusClient("rag_wikipedia_mini.db")

# Create the Collection with Collection Name = "rag_mini". Make sure you define the schema variable while creating the collection
try:
    client.drop_collection("rag_mini")
except:
    pass

client.create_collection(
    collection_name="rag_mini",
    schema=schema
)

**Convert your Pandas Dataframe to a list of dictionaries**
- The Dictionary at least have 3 keys [id, passage, embedding]

In [ ]:
# ---- Improved rag_data
rag_data = []
for i, passage_text in enumerate(texts):
    vec = embeddings[i].astype("float32").ravel().tolist()
    vec = [float(x) for x in vec]
    rag_data.append({
        "id": int(i),
        "passage": passage_text,
        "embedding": vec
    })


In [ ]:
# Code to insert the data to your DB
res = client.insert(collection_name="rag_mini", data=rag_data)

print(res)

{'insert_count': 11255, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215

In [ ]:
# === Build index on the 'embedding' field, then load the collection ===
index_params = MilvusClient.prepare_index_params()
index_params.add_index(
    field_name="embedding",
    index_type="IVF_FLAT",
    metric_type="COSINE",
    params={"nlist": 128}
)

try:
    client.create_index(
        collection_name="rag_mini",
        index_params=index_params
    )
    print("Index created successfully!")
except Exception as e:
    print(f"Index creation result: {e}")

client.load_collection("rag_mini")
print("Collection loaded into memory")


Index created successfully!
Collection loaded into memory


- Do a Sanity Check on your database

**Do not delete the below line during your submission**

In [ ]:
print("Entity count:", client.get_collection_stats("rag_mini")["row_count"])
print("Collection schema:", client.describe_collection("rag_mini"))

Entity count: 11255
Collection schema: {'collection_name': 'rag_mini', 'auto_id': False, 'num_shards': 0, 'description': 'Mini Wikipedia passages with embeddings', 'fields': [{'field_id': 100, 'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'params': {}, 'is_primary': True}, {'field_id': 101, 'name': 'passage', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'field_id': 102, 'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'functions': [], 'aliases': [], 'collection_id': 0, 'consistency_level': 0, 'properties': {}, 'num_partitions': 0, 'enable_dynamic_field': False}


## Steps to Fetch Results
- Read the Question Dataset
- Clean the Question Dataset if necessary (Drop Questions with NaN etc.)
- Convert Each Query to a Vector Embedding (Use the same embedding model you used to embed your document)
- Try for a Single Question First
- Load Collection into Memory after creating Index for Search on your embedding field (This is an essential step before you can search in your db)
- Search and Fetch Top N Results

In [ ]:
import pandas as pd

queries = pd.read_parquet("hf://datasets/rag-datasets/rag-mini-wikipedia/data/test.parquet/part.0.parquet")
queries

,question,answer
id,,
0,Was Abraham Lincoln the sixteenth President of...,yes
2,Did Lincoln sign the National Banking Act of 1...,yes
4,Did his mother die of pneumonia?,no
6,How many long was Lincoln's formal education?,18 months
8,When did Lincoln begin his political career?,1832
...,...,...
1710,Was Wilson president of the American Political...,Yes
1711,Did he not cast his ballot for John M. Palmer ...,Yes
1712,Did Wilson not spend 1914 through the beginnin...,Yes


## RAG Response for a Single Query

In [ ]:
# Load the LLM Model you want to use
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

rag_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


## Generate Responses for all the Queries in the Dataset

In [212]:
# Your Code Here
eval_df = queries

STRATEGIES = {
    "instruction": (
        "Answer the following question using ONLY the given context."
        "Do NOT add outside knowledge."
        "Provide the answer in one clear, concise sentence."
        "If the context is insufficient, reply exactly: I don't know."
    ),
    "cot": (
        "Let's think step by step. First, read the question and context and extract the key entities, numbers, dates, and relations."
        "Then, do a step-by-step reasoning, using ONLY the given context and no outside knowledge. "
        "If the context is insufficient to answer, reply exactly: I don't know."
        "Finally, output the concise answer in one sentence."
    ),
    "persona": (
       "You are a historian specializing in U.S. political history."
       "Your expertise ensures that answers are accurate, well-grounded, and concise."
    ),
}

In [213]:
def build_prompt(strategy, question, contexts):
    ctx_text = "\n\n".join(f"[{i}] {c}" for i, c in enumerate(contexts, 1))
    k = len(contexts)

    guide = (
        "You are a QA model that MUST output exactly ONE line in this schema:\n"
        "Answer: <concise answer> [<indices>]\n"
        "Rules:\n"
        f"- Valid indices are integers from 1 to {k} ONLY (matching the context list below).\n"
        "- The line MUST end with a space and a bracketed list like [1] or [1,3].\n"
        "- For yes/no questions, output 'Answer: yes [i]' or 'Answer: no [i]'.\n"
        "- If insufficient, output 'Answer: I don't know [i]' where [i] is the most relevant context.\n"
        "- Do NOT add extra text, newlines, or explanations.\n"
    )

    fewshot = (
        "Context:\n"
        "[1] The capital of France is Paris.\n"
        "[2] Berlin is the capital of Germany.\n"
        "Q: Is Paris the capital of France?\n"
        "Answer: yes [1]\n"
        "----\n"
        "Context:\n"
        "[1] The iPhone 15 uses a USB-C connector.\n"
        "[2] The iPhone X uses Lightning.\n"
        "Q: Which iPhone model first switched to USB-C?\n"
        "Answer: iPhone 15 [1]\n"
        "----\n"
    )

    base = STRATEGIES[strategy]
    # base = STRATEGIES[strategy].format(question=question, context=ctx_text)

    return f"{guide}\n{fewshot}\n{base}\n\nContext:\n{ctx_text}\nQ: {question}\nAnswer:"



In [214]:
if 'id' not in eval_df.columns:
    eval_df = eval_df.reset_index(drop=True)
    eval_df['id'] = eval_df.index.astype(str)
eval_df.head()

,question,answer,id
0,Was Abraham Lincoln the sixteenth President of...,yes,0
1,Did Lincoln sign the National Banking Act of 1...,yes,1
2,Did his mother die of pneumonia?,no,2
3,How many long was Lincoln's formal education?,18 months,3
4,When did Lincoln begin his political career?,1832,4


In [215]:
def retrieve_topk_context(question, k=1):
    qvec = embedding_model.encode([question], convert_to_numpy=True, show_progress_bar=False).astype("float32")
    qlist = [float(x) for x in qvec.reshape(-1).tolist()]
    output_ = client.search(
        collection_name="rag_mini",
        data=[qlist],
        anns_field="embedding",
        search_params={"metric_type": "COSINE", "params": {"nprobe": 10}},
        limit=k,
        output_fields=["id", "passage"],
    )
    return [hit["entity"]["passage"] for hit in output_[0]]

In [216]:
from tqdm.auto import tqdm
import time

all_preds = {}

for strategy in STRATEGIES.keys():
    preds = []
    print(f"Running strategy: {strategy}")
    for _, row in tqdm(eval_df.iterrows(), total=len(eval_df), desc=f"{strategy}"):
        q_id = str(row["id"])
        q = str(row["question"])
        contexts = retrieve_topk_context(q, k=1)
        # rewrites = rewrite_query(q, n=1)
        # contexts = retrieve_with_rewrite(q, k=1)
        prompt = build_prompt(strategy, q, contexts)
        # prompt = build_prompt(strategy, q, "\n\n".join(contexts), rewrites=rewrites)

        out = rag_pipeline(
            prompt,
            max_new_tokens=128,
            min_new_tokens=2,
            do_sample=False,
            num_beams=1
        )[0]["generated_text"].strip()
        preds.append({"id": q_id, "prediction_text": out})
    all_preds[strategy] = preds

references = [
    {"id": str(r["id"]), "answers": {"text": [str(r["answer"])], "answer_start": [0]}}
    for _, r in eval_df.iterrows()
]

print("Done. Strategies:", list(all_preds.keys()), "| #items =", len(references))


Running strategy: instruction


instruction:   0%|          | 0/918 [00:00<?, ?it/s]

Running strategy: cot


cot:   0%|          | 0/918 [00:00<?, ?it/s]

Running strategy: persona


persona:   0%|          | 0/918 [00:00<?, ?it/s]

Done. Strategies: ['instruction', 'cot', 'persona'] | #items = 918


## Finding out the Basic QA Metrics (F1 score, EM score)

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00


In [205]:
import re
_CITE_TAIL = re.compile(r"\s*\[\s*\d+(?:\s*,\s*\d+)*\s*\]\s*$")

def extract_final_answer(s):
    if "Answer:" in s:
        ans = s.split("Answer:", 1)[1].strip()
        ans = _CITE_TAIL.sub("", ans).strip()
        return ans
    return s.strip()


In [217]:
import evaluate, pandas as pd
metric = evaluate.load("squad")
# Evaluate multiple strategies with SQuAD EM/F1: clean predictions to compute metrics to rank by F1/EM

def _clean_preds_for_metric(preds, rows_df):
  # Normalize prediction schema and strip CoT using extract_final_answer()
    cleaned = []
    for p in preds:
        q = str(rows_df.loc[int(p["id"]), "question"]) if "question" in rows_df.columns else ""
        pred_text = extract_final_answer(p["prediction_text"])
        cleaned.append({"id": str(p["id"]), "prediction_text": pred_text})
    return cleaned

references_clean = [{"id": str(r["id"]), "answers": r["answers"]} for r in references]

rows = []
for strat, preds in all_preds.items():
    preds_clean = _clean_preds_for_metric(preds, eval_df)
    scores = metric.compute(predictions=preds_clean, references=references_clean)
    rows.append({"strategy": strat, "EM": scores["exact_match"], "F1": scores["f1"]})

result_df = pd.DataFrame(rows).sort_values(["F1","EM"], ascending=False).reset_index(drop=True)
display(result_df)

best = result_df.iloc[0]
print(f"Best strategy: {best['strategy']} | EM={best['EM']:.2f} | F1={best['F1']:.2f}")


,strategy,EM,F1
0,instruction,43.790850,49.658188
1,persona,43.028322,48.559629
2,cot,24.618736,32.762037


Best strategy: instruction | EM=43.79 | F1=49.66


# Advanced Evaluation using RAGAs

In [222]:
pip install openai ragas langchain-openai

In [224]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


In [225]:
BEST_STRATEGY = result_df.sort_values(["F1","EM"], ascending=False).iloc[0]["strategy"]
print("Use strategy for RAGAs:", BEST_STRATEGY)


Use strategy for RAGAs: instruction


In [226]:
def build_records(eval_df, preds_for_strategy, get_contexts_fn):
    # Build evaluation records by combining questions, predicted answers,
    # retrieved contexts, and ground-truth answers
    id2pred = {str(p["id"]): p["prediction_text"] for p in preds_for_strategy}


    records = []
    for _, r in eval_df.iterrows():
        q_id = str(r["id"])
        q = str(r["question"])
        gt = str(r["answer"])

        ctxs = get_contexts_fn(q)
        if isinstance(ctxs, str):
            ctxs = [ctxs]

        ans = id2pred.get(q_id, "")
        records.append({
            "question": q,
            "answer": ans,
            "contexts": [str(c) for c in ctxs],
            "ground_truth": gt
        })
    return records


In [227]:
adv_records = build_records(
    eval_df=eval_df,
    preds_for_strategy=all_preds[BEST_STRATEGY],
    get_contexts_fn=lambda q: retrieve_topk_context(q, k=1)
)

print(len(adv_records))

918


In [ ]:
data = {
    "question": ... ,                     # Question
    "answer": ... ,                       # Generated Answer
    "contexts": ... ,                     # Context you pass in. You can just use top-1 here
    "ground_truths": ...                  # Reference Answer in the dataset (Human annotated)
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [ ]:
# Pass the dataset above to the evaluate method in RAGAs
# Your code here
...

In [253]:
import random
random.seed(42)

N_SAMPLE_EACH = 10
MAX_CTX_CHARS = 600

def _sample_and_clip(records, n, max_chars):
    if n and n < len(records):
        records = random.sample(records, n)
    for r in records:
        r["contexts"] = [str(c)[:max_chars] for c in r.get("contexts", [])]
    return records

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, context_precision, context_recall, answer_relevancy

# Run RAGAs evaluation on a set of records and return detailed results + averaged summary

def run_ragas(records, llm):
    ds = Dataset.from_list(records)
    res = evaluate(
        dataset=ds,
        metrics=[faithfulness, context_precision, context_recall, answer_relevancy],
        llm=llm
    )
    df = res.to_pandas()
    summary = df[["faithfulness","context_precision","context_recall","answer_relevancy"]].mean()
    return df, summary.to_frame(name="score")

In [255]:
import random, pandas as pd, os
random.seed(42)

N_SAMPLE_EACH = 8
MAX_CTX_CHARS = 600

# Randomly sample records and truncate contexts for faster evaluation
def _sample_and_clip(records, n, max_chars):
    recs = records if n >= len(records) else random.sample(records, n)
    for r in recs:
        r["contexts"] = [str(c)[:max_chars] for c in r.get("contexts", [])]
    return recs

naive_fast = _sample_and_clip(naive_records, N_SAMPLE_EACH, MAX_CTX_CHARS)
adv_fast   = _sample_and_clip(adv_records,   N_SAMPLE_EACH, MAX_CTX_CHARS)

print(f"Samples -> naive: {len(naive_fast)}, advanced: {len(adv_fast)}")

# Run evaluation on sampled subsets and save results
df_naive, sum_naive = run_ragas(naive_fast, llm, batch=3, sleep_between_batches=10)
df_adv,   sum_adv   = run_ragas(adv_fast,   llm, batch=3, sleep_between_batches=10)

os.makedirs("results", exist_ok=True)

df_naive.to_csv("results/ragas_naive_detail_sampled.csv", index=False)
df_adv.to_csv("results/ragas_advanced_detail_sampled.csv", index=False)


# Compare metrics between naive and advanced, compute delta improvements
sum_naive = sum_naive.rename(columns={"score":"naive"})
sum_adv   = sum_adv.rename(columns={"score":"advanced"})

comparison = (
    pd.concat([sum_naive, sum_adv], axis=1)
      .assign(delta=lambda x: x["advanced"] - x["naive"])
      .sort_values("delta", ascending=False)
)

comparison.to_csv("results/ragas_comparison_summary_sampled.csv")
display(comparison)

def pct(x): return f"{x*100:.1f}%"
print("\n==== RAGAs Summary (Advanced - Naive) on Sampled Data ====")
for m in ["faithfulness","context_precision","context_recall","answer_relevancy"]:
    dv = float(comparison.loc[m, "delta"])
    print(f"{m}: {dv:+.4f} ({pct(dv)})")

print("\nSaved:")
print(" - results/ragas_naive_detail_sampled.csv")
print(" - results/ragas_advanced_detail_sampled.csv")
print(" - results/ragas_comparison_summary_sampled.csv")


Samples -> naive: 8, advanced: 8


Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[0]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[4]: TimeoutError()


Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[0]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[4]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[5]: TimeoutError()


...

ERROR:ragas.executor:Exception raised in Job[1]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[3]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[4]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[5]: TimeoutError()


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[1]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[3]: TimeoutError()


,naive,advanced,delta
context_precision,0.250000,0.800000,0.550000
faithfulness,0.500000,1.000000,0.500000
context_recall,0.166667,0.000000,-0.166667
answer_relevancy,0.783891,0.544308,-0.239582



==== RAGAs Summary (Advanced - Naive) on Sampled Data ====
faithfulness: +0.5000 (50.0%)
context_precision: +0.5500 (55.0%)
context_recall: -0.1667 (-16.7%)
answer_relevancy: -0.2396 (-24.0%)

Saved:
 - results/ragas_naive_detail_sampled.csv
 - results/ragas_advanced_detail_sampled.csv
 - results/ragas_comparison_summary_sampled.csv


## Experiments

In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, context_precision, context_recall, answer_relevancy
from concurrent.futures import ThreadPoolExecutor, as_completed
from datasets import Dataset
import pandas as pd, time
from openai import APIConnectionError, RateLimitError

def run_ragas_fast(records, pipe, max_workers=3, chunk_size=12, retries=3, backoff=5):
    metrics_list = [faithfulness, context_precision, context_recall, answer_relevancy]

    def eval_chunk(chunk):
        for attempt in range(retries):
            try:
                ds = Dataset.from_list(chunk)
                res = evaluate(dataset=ds, metrics=metrics_list, llm=llm)
                return res.to_pandas()
            except (APIConnectionError, RateLimitError) as e:
                wait = backoff * (attempt + 1)
                print(f"[Retry] Chunk failed with {type(e).__name__}, attempt {attempt+1}/{retries}, sleeping {wait}s...")
                time.sleep(wait)
        raise

    chunks = [records[i:i+chunk_size] for i in range(0, len(records), chunk_size)]
    dfs = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(eval_chunk, c) for c in chunks]
        for fut in as_completed(futures):
            dfs.append(fut.result())

    df = pd.concat(dfs, ignore_index=True)
    summary = df[["faithfulness","context_precision","context_recall","answer_relevancy"]].mean()
    summary = summary.to_frame(name=pipe)
    return df, summary


In [ ]:
df_naive, sum_naive = run_ragas_fast(naive_records, "naive", max_workers=4, chunk_size=12)
df_adv,   sum_adv   = run_ragas_fast(adv_records,   "advanced", max_workers=4, chunk_size=12)

In [ ]:
pip install backoff

In [ ]:
from datasets import Dataset
import pandas as pd
import time, random, json, os
from concurrent.futures import ThreadPoolExecutor, as_completed
import backoff

from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from langchain_openai import ChatOpenAI

MODEL_NAME = "gpt-4o-mini"
DEFAULT_TIMEOUT_S = 60
N_WORKERS  = 4
SHARD_SIZE = 8
RETRIES    = 3
METRICS    = [faithfulness, answer_relevancy, context_precision, context_recall]

def _mk_llm():
    return ChatOpenAI(model=MODEL_NAME, temperature=0, timeout=DEFAULT_TIMEOUT_S)

def _chunks(lst, k):
    for i in range(0, len(lst), k):
        yield lst[i:i+k]

def _to_hf_dataset(recs):
    df = pd.DataFrame(recs)
    needed = ["question", "answer", "contexts", "ground_truth"]
    missing = [c for c in needed if c not in df.columns]
    if missing:
        raise ValueError(f"records 缺少列: {missing}")
    return Dataset.from_pandas(df[needed], preserve_index=False)

def _jitter(base=1.0, spread=0.75):
    return base + random.random() * spread

@backoff.on_exception(backoff.expo, Exception, max_tries=6, jitter=_jitter)
def _eval_shard(records_subset):
    ds = _to_hf_dataset(records_subset)
    llm_local = _mk_llm()
    res = evaluate(dataset=ds, metrics=METRICS, llm=llm_local)
    try:
        return res.to_pandas()
    except Exception:
        return pd.DataFrame(res)

def run_ragas(records, run_name, max_workers=N_WORKERS, shard_size=SHARD_SIZE, attempts=RETRIES):
    if not records:
        raise ValueError(f"{run_name}: records 为空。")
    shards = list(_chunks(records, shard_size))
    all_dfs, remaining = [], shards

    for attempt in range(1, attempts + 1):
        if not remaining: break
        print(f"[{run_name}] Attempt {attempt}/{attempts} - shards: {len(remaining)}")
        failed = []
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            futs = {ex.submit(_eval_shard, s): i for i, s in enumerate(remaining)}
            for fut in as_completed(futs):
                try:
                    all_dfs.append(fut.result())
                except Exception as e:
                    idx = futs[fut]
                    print(f"[{run_name}] shard {idx} failed: {type(e).__name__}: {e}")
                    failed.append(remaining[idx])
        if failed:
            cool = 5 + 3 * attempt
            print(f"[{run_name}] {len(failed)} shard(s) failed; cool down {cool}s…")
            time.sleep(cool)
        remaining = failed

    if not all_dfs:
        raise RuntimeError(f"{run_name}: no shard succeeded.")
    df = pd.concat(all_dfs, ignore_index=True)

    agg = {
        "faithfulness":      df["faithfulness"].mean(),
        "answer_relevancy":  df["answer_relevancy"].mean(),
        "context_precision": df["context_precision"].mean(),
        "context_recall":    df["context_recall"].mean(),
        "n":                 len(df),
    }
    os.makedirs("results", exist_ok=True)
    df.to_csv(f"results/ragas_{run_name}.csv", index=False)
    with open(f"results/ragas_{run_name}_summary.json", "w") as f:
        json.dump(agg, f, indent=2)
    print(f"[{run_name}] Summary:", agg)
    return agg

naive_summary    = run_ragas(naive_records,    run_name="naive")
advanced_summary = run_ragas(adv_records, run_name="advanced")

cmp = pd.DataFrame([naive_summary, advanced_summary], index=["naive", "advanced"])[
    ["faithfulness","answer_relevancy","context_precision","context_recall","n"]
]
print("\n=== RAGAs comparison (mean over samples) ===")
print(cmp)
print("\nDelta (advanced - naive):")
print(cmp.loc["advanced"].drop("n") - cmp.loc["naive"].drop("n"))
